
---

### Overview

Our system is designed to recommend the best financial advisor (agent) for a new client by combining several techniques. In essence, we use three different “views” of the data, then combine them with a final machine learning model. The three parts are:

1. **Metric Learning (Client Similarity)**
2. **Agent–Product Rating**
3. **Agent–Client Background Similarity**

Finally, we merge all these features and train an XGBoost classifier to output a final ranking score for each agent.

---

### 1. Metric Learning (Client Similarity)

- **Goal:** Learn a way to measure how similar two clients are.
- **How:**  
  - We preprocessed all our historical client data (using one‑hot encoding for categorical data and scaling for numerical data).
  - We then built positive and negative pairs of clients:
    - **Positive pair:** Two clients who have been served by the same agent.
    - **Negative pair:** Two clients who have never been served by the same agent.
  - Using these pairs, we trained a **Siamese neural network**.  
    - This network learns to embed (or convert) each client into a 16-dimensional vector where similar clients are close together.
  - Finally, we precompute the embeddings for all historical clients so that for a new client we can quickly find the most similar ones.

---

### 2. Agent–Product Rating

- **Goal:** Identify which financial products are most suitable for the new client, then measure how experienced each agent is with those products.
- **How:**  
  - For the new client, we use our Siamese network to find the top similar clients.
  - We then look at the policies (sales) of those similar clients to see which products were most frequently purchased. This gives us a **suitability score** for each product.
  - Next, for each agent we look at their historical product conversion rates and product expertise (which we turned into dummy variables).  
    - For each top product, we calculate a score based on a weighted combination of the agent’s conversion rate and expertise.
  - We sum these scores across the top products to get an overall **agent–product rating**.  
  - Finally, we normalize these ratings.

---

### 3. Agent–Client Background Similarity

- **Goal:** Measure how similar an agent's typical client profile is to our new client using basic demographics.
- **How:**  
  - We use only three features from both sides: age, gender, and marital status.
  - For agents, we use their own demographic information (e.g., agent_age, agent_female, and agent_marital).
  - For the client, we use their age, client_female, and client_marital.
  - We compute the cosine similarity between the new client’s demographic vector and each agent’s demographic vector.  
    - A higher cosine similarity means the agent is more experienced with a demographic profile similar to the new client.

---

### 4. Final Model & Ranking

- **Goal:** Combine the above three components into a single score that tells us which agent is most likely to succeed with the new client.
- **How:**  
  - We merge features from the agent–client similarity (like match_count, gender alignment, and age alignment), the agent–product rating, and the background similarity scores.
  - We define a binary target for training: if an agent has ever served a client similar to the new client, that’s a success (target=1); otherwise, it's 0.
  - We split our data into 80% training and 20% testing.
  - Then, we train an XGBoost classifier (a powerful gradient boosting model) on these features to predict the likelihood of a successful match.
  - The model outputs a probability score (final_score) for each agent. Agents are ranked based on this final_score.

---

### Summary

- **Metric Learning:** Finds a compact representation of client data so we can identify similar clients.
- **Agent–Product Rating:** Determines which products are most suitable for the new client and measures each agent’s expertise and success with those products.
- **Background Similarity:** Compares the new client’s demographics to the agent’s typical client profile.
- **Final Ranking:** Combines all of the above with a machine learning model (XGBoost) to rank agents.

This multi-step approach allows us to incorporate both behavioral (what products are bought, which agents have success) and demographic information to provide a robust recommendation for matching a new client with the best advisor.

---


### Load Libraries

In [ ]:
import pandas as pd
import numpy as np
from itertools import combinations
from tensorflow.keras.layers import Input, Dense, Lambda
from tensorflow.keras.models import Model
from tensorflow.keras import backend as K
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, mean_squared_error, roc_curve
import xgboost as xgb
from xgboost import XGBRegressor, XGBClassifier
from sklearn.metrics.pairwise import cosine_similarity
import matplotlib.pyplot as plt
import ast
from sklearn.preprocessing import MultiLabelBinarizer

### Agent Dataset Cleaning

In [ ]:
############################################
# 1. Load the Agent Data
############################################
df_agents = pd.read_parquet('nus_agent_info_df.parquet')

############################################
# 2. Create "agent_female" Column
############################################
# Here, we assume that the 'agent_gender' column contains values like "F", "Female", "M", "Male", etc.
# We define agent_female = 1 if the gender (after stripping and converting to uppercase) starts with 'F'; else 0.
df_agents['agent_female'] = df_agents['agent_gender'].apply(lambda x: 1 if isinstance(x, str) and x.strip().upper().startswith('F') else 0)

############################################
# 3. Drop Unnecessary Columns
############################################
df_agents = df_agents.drop(columns=['agent_gender', 'pct_SX0_unknown', 'cluster'])

############################################
# 4. Convert "agent_product_expertise" into Dummy Variables
############################################
# The 'agent_product_expertise' column contains lists (or string representations of lists) of products.
# First, define a helper to parse each value into a list.
def parse_expertise(x):
    if isinstance(x, str):
        try:
            return ast.literal_eval(x)
        except Exception:
            return []
    elif isinstance(x, list):
        return x
    else:
        return []

df_agents['agent_product_expertise'] = df_agents['agent_product_expertise'].apply(parse_expertise)

# Use MultiLabelBinarizer to create dummy columns.
mlb = MultiLabelBinarizer()
expertise_array = mlb.fit_transform(df_agents['agent_product_expertise'])
expertise_dummies = pd.DataFrame(
    expertise_array,
    columns=[f"agent_expertise_{p}" for p in mlb.classes_],
    index=df_agents.index
)

# Define the desired products (subset) as given.
desired_products = ['prod_0', 'prod_2', 'prod_4', 'prod_6', 'prod_7', 'prod_8', 'prod_9']
desired_columns = [f"agent_expertise_{p}" for p in desired_products]
# Reindex to ensure all desired columns exist (fill missing with zeros).
expertise_dummies = expertise_dummies.reindex(columns=desired_columns, fill_value=0)

# Concatenate the dummies with the original DataFrame and drop the original expertise column.
df_agents = pd.concat([df_agents, expertise_dummies], axis=1)
df_agents = df_agents.drop(columns=['agent_product_expertise'])

############################################
# 5. Check Age Group Percentage Columns
############################################
age_group_cols = ['pct_AG01_lt20', 'pct_AG02_20to24', 'pct_AG03_25to29',
                  'pct_AG04_30to34', 'pct_AG05_35to39', 'pct_AG06_40to44',
                  'pct_AG07_45to49', 'pct_AG08_50to54', 'pct_AG09_55to59',
                  'pct_AG10_60up']

# Calculate the row sums.
row_sums = df_agents[age_group_cols].sum(axis=1)
# If the sums are not approximately 1 (within a tolerance), print a warning.
if not np.allclose(row_sums, 1, atol=0.01):
    print("Warning: Not all rows in the age group columns sum to 1.")
    # Optionally, you could normalize these rows. For example, uncomment the following:
    # df_agents[age_group_cols] = df_agents[age_group_cols].div(row_sums, axis=0)
else:
    print("All rows in age group columns sum to 1 within tolerance.")

############################################
# 6. Change 'agent_marital' to Full Names
############################################
# Suppose the current 'agent_marital' column contains abbreviated values:
# 'M' for Married, 'S' for Single, 'U' for Unknown, 'D' for Divorced, 'W' for Widowed.
marital_mapping = {
    'M': 'Married',
    'S': 'Single',
    'U': 'Unknown',
    'D': 'Divorced',
    'W': 'Widowed'
}
df_agents['agent_marital'] = df_agents['agent_marital'].map(marital_mapping)

############################################
# 7. Check the Final DataFrame
############################################
df_agents.head()


### Client Dataset Cleaning

In [ ]:
df_clients = pd.read_parquet('nus_client_info_df.parquet')
# Change Data Types
df_clients['family_size'] = df_clients['family_size'].astype('Int64')
df_clients['economic_status'] = df_clients['economic_status'].astype('Int64')
df_clients['household_size'] = df_clients['household_size'].astype('Int64')
# Remove invalid postal code data
df_clients = df_clients[df_clients['cltpcode'] <= '900000']
df_clients = df_clients.dropna(subset=['cltdob', 'race_desc_map'])
df_clients['age'] = 2025 - df_clients['cltdob'].dt.year
df_clients = df_clients.dropna(subset=['economic_status'])
df_clients = df_clients.rename(columns={'secuityno': 'securityno'})
df_clients.loc[df_clients['securityno'] == 'CIN:11715', 'household_size_grp'] = 'HH3_80to100'
df_clients.loc[df_clients['securityno'] == 'CIN:11715', 'family_size_grp'] = 'FS4_60to80'
income_mapping = {
    'HH1_lt40': 1,
    'HH2_40to80': 2,
    'HH3_80to100': 3,
    'HH4_100to120': 4,
    'HH5_120up': 5
}

df_clients['household_size_grp'] = df_clients['household_size_grp'].map(income_mapping)
# Map family size groups

family_mapping = {
    'FS1_lt20': 1,
    'FS2_20to40': 2,
    'FS3_40to60': 3,
    'FS4_60to80': 4,
    'FS5_80up': 5
}


df_clients['family_size_grp'] = df_clients['family_size_grp'].map(family_mapping)
# Remove customer with marital status as P
df_clients = df_clients[df_clients['marryd'] != 'P']
# Remove columns
df_clients.drop(['cltdob', 'race_desc_map'], axis=1, inplace=True)

df_clients['client_marital'] = df_clients['marryd'].map(marital_mapping)
# Map postal codes
postal_districts = {'01': '01', '02': '01', '03': '01', '04': '01', '05': '01', '06': '01', '07': '02', '08': '02', '14': '03', '15': '03', '16': '03', '09': '04', '10': '04', '11': '05', '12': '05', '13': '05', '17': '06', '18': '07', '19': '07', '20': '08', '21': '08', '22': '09', '23': '09', '24': '10', '25': '10', '26': '10', '27': '10', '28': '11', '29': '11', '30': '11', '31': '12', '32': '12', '33': '12', '34': '13', '35': '13', '36': '13', '37': '13', '38': '14', '39': '14', '40': '14', '41': '14', '42': '15', '43': '15', '44': '15', '45': '15', '46': '16', '47': '16', '48': '16', '49': '17', '50': '17', '81': '17', '51': '18', '52': '18', '53': '19', '54': '19', '55': '19', '82': '19', '56': '20', '57': '20', '58': '21', '59': '21', '60': '22', '61': '22', '62': '22', '63': '22', '64': '22', '65': '23', '66': '23', '67': '23', '68': '23', '69': '24', '70': '24', '71': '24', '72': '25', '73': '25', '77': '26', '78': '26', '75': '27', '76': '27', '79': '28', '80': '28'}

# # Convert the postal code to an integer, then extract the first two digits
# df_clients['cltpcode'] = df_clients['cltpcode'].astype(int)

# # Extract the first two digits by integer division
# df_clients['district_code'] = df_clients['cltpcode'] // 10000
# df_clients['district_code'] = df_clients['district_code'].astype(str)

# # Map the district code to the corresponding district
# df_clients['district'] = df_clients['district_code'].map(postal_districts)
df_clients['district'] = df_clients['cltpcode'].str[:2].map(postal_districts)
df_clients['client_female'] = df_clients['cltsex'].apply(lambda x: 1 if isinstance(x, str) and x.strip().upper().startswith('F') else 0)
# Remove columns
df_clients.drop(['cltsex', 'marryd', 'cltpcode','household_size_grp' , 'family_size_grp'], axis=1, inplace=True)
df_clients

### Policy Dataset Cleaning

In [ ]:
# Load the policy data from a parquet file.
df_policies = pd.read_parquet("nus_policy_info_df.parquet")

# Ensure that 'occdate' is in datetime format.
df_policies['occdate'] = pd.to_datetime(df_policies['occdate'])

# Create a new column 'policy_age' as the difference between 2025 and the year of the occdate.
df_policies['policy_age'] = 2025 - df_policies['occdate'].dt.year

# Drop the unnecessary columns.
df_policies = df_policies.drop(columns=['occdate', 'flg_main', 'flg_rider', 'flg_inforce', 'flg_cancel', 'flg_converted', 'product_grp'])
df_policies.rename(columns={'secuityno': 'securityno'}, inplace=True)

# Optionally, inspect the resulting DataFrame.
df_policies.head()


### Model

In [ ]:
############################################
# SETUP & NEW CLIENT DATA
############################################
new_client_data = {
    'securityno': 'NEW:001',
    'client_marital': 'Married',
    'economic_status': 52,
    'household_size': 4,
    'family_size': 5,
    'age': 37,
    'district': 25,
    'client_female': 0
}

In [ ]:
############################################
# SECTION 1: AGENT–SIMILAR CLIENTS (METRIC LEARNING)
############################################

# 1.2 Preprocess Client Data

# Define categorical and numerical columns for feature processing
categorical_cols = ['client_marital', 'district']
numerical_cols = ['economic_status', 'age', 'client_female', 'household_size', 'family_size']

# Convert categorical variables into one-hot encoded features
df_client_encoded = pd.get_dummies(df_clients, columns=categorical_cols)

# Standardize (scale) numerical columns to have zero mean and unit variance
scaler = StandardScaler()
df_client_encoded[numerical_cols] = scaler.fit_transform(df_client_encoded[numerical_cols])

# Extract feature column names, excluding the unique client identifier ('securityno')
client_feature_cols = [col for col in df_client_encoded.columns if col != 'securityno']

# Convert client data into a dictionary format where each client's features are stored under their security number
client_features = df_client_encoded.set_index('securityno')[client_feature_cols].to_dict('index')

# 1.3 Build Pairs for Metric Learning

# Filter policies to include only clients present in the processed dataset
df_policies_filtered = df_policies[df_policies['securityno'].isin(df_client_encoded['securityno'])]

# Create a dictionary that maps each client to the set of agents they have worked with
client_agents = df_policies_filtered.groupby('securityno')['agntnum'].apply(set).to_dict()

# Create a set to store positive pairs (clients who were served by the same agent)
positive_pairs = set()

# Iterate over each agent and find all unique client pairs that the agent has served
for agent, group in df_policies_filtered.groupby('agntnum'):
    clients = group['securityno'].unique()
    clients = [c for c in clients if c in client_features]  # Ensure clients are in the dataset
    for pair in combinations(clients, 2):  # Create all possible client pairs
        positive_pairs.add(tuple(sorted(pair)))  # Store the pair in a sorted format

# Get a list of all client IDs
all_clients = list(client_features.keys())

# Generate the same number of negative pairs (clients who never shared an agent)
num_negatives = len(positive_pairs)
negative_pairs = set()

# Randomly create negative pairs until we have the same number as positive pairs
while len(negative_pairs) < num_negatives:
    c1, c2 = np.random.choice(all_clients, 2, replace=False)  # Select two random clients
    pair = tuple(sorted((c1, c2)))  # Ensure consistent ordering
    agents1 = client_agents.get(c1, set())  # Get agents who served client 1
    agents2 = client_agents.get(c2, set())  # Get agents who served client 2

    # Ensure the two clients have never been served by the same agent
    if len(agents1.intersection(agents2)) == 0:
        negative_pairs.add(pair)

# Combine positive and negative pairs into a single dataset
pairs = list(positive_pairs) + list(negative_pairs)

# Assign labels: 1 for positive pairs (similar clients), 0 for negative pairs (dissimilar clients)
labels = [1]*len(positive_pairs) + [0]*len(negative_pairs)

# Print the number of pairs created
print(f"Number of pairs: {len(pairs)} (Positive: {len(positive_pairs)}, Negative: {len(negative_pairs)})")

# Function to extract client features given a client ID
def get_feature(securityno):
    return np.array(list(client_features[securityno].values()))

# Create input arrays for model training (features for each client in the pair)
X1 = np.array([get_feature(c1) for (c1, c2) in pairs])
X2 = np.array([get_feature(c2) for (c1, c2) in pairs])
y = np.array(labels)  # Labels for similarity classification

# 1.4 Build the Siamese Network

# Define the input dimension based on the number of client features
input_dim = X1.shape[1]

# Function to create the base neural network for feature embedding
def create_base_network(input_dim):
    inp = Input(shape=(input_dim,))
    x = Dense(128, activation='relu')(inp)
    x = Dense(64, activation='relu')(x)
    x = Dense(32, activation='relu')(x)
    x = Dense(16, activation='relu')(x)  # Final embedding size is 16-dimensional
    return Model(inp, x)

# Create the base network
base_network = create_base_network(input_dim)

# Define inputs for the Siamese network (two clients to compare)
input_a = Input(shape=(input_dim,))
input_b = Input(shape=(input_dim,))

# Process both inputs using the same network (shared weights)
processed_a = base_network(input_a)
processed_b = base_network(input_b)

# Function to compute Euclidean distance between two client embeddings
def euclidean_distance(vects):
    x, y = vects
    sum_square = K.sum(K.square(x - y), axis=1, keepdims=True)
    return K.sqrt(K.maximum(sum_square, K.epsilon()))

# Compute the Euclidean distance between the two processed inputs
distance = Lambda(euclidean_distance, output_shape=lambda shapes: (shapes[0][0], 1))([processed_a, processed_b])

# Create the Siamese network model
model = Model([input_a, input_b], distance)

# Define contrastive loss function for training
def contrastive_loss(y_true, y_pred):
    margin = 1  # Defines the decision boundary
    square_pred = K.square(y_pred)  # Squared distance for similar pairs
    margin_square = K.square(K.maximum(margin - y_pred, 0))  # Distance for dissimilar pairs
    return K.mean(y_true * square_pred + (1 - y_true) * margin_square)  # Compute loss

# Compile the model with contrastive loss and Adam optimizer
model.compile(loss=contrastive_loss, optimizer='adam')

# Display model summary
model.summary()

# Train the model using labeled client pairs
model.fit([X1, X2], y, batch_size=128, epochs=10, validation_split=0.2)

# Generate embeddings for all clients in the dataset using the trained model
client_ids = df_client_encoded['securityno'].tolist()
client_data = df_client_encoded[client_feature_cols].astype('float32').values
client_embeddings = base_network.predict(client_data)


In [ ]:
############################################
# SECTION 2: AGENT RECOMMENDATION (AGENT–PRODUCT & AGENT–CLIENT)
############################################
def preprocess_new_client(new_client_data):
    processed_client = {}
    processed_client['economic_status'] = (new_client_data['economic_status'] - scaler.mean_[0]) / scaler.scale_[0]
    processed_client['age'] = (new_client_data['age'] - scaler.mean_[1]) / scaler.scale_[1]
    processed_client['client_female'] = (new_client_data['client_female'] - scaler.mean_[2]) / scaler.scale_[2]
    processed_client['household_size'] = (new_client_data['household_size'] - scaler.mean_[3]) / scaler.scale_[3]
    processed_client['family_size'] = (new_client_data['family_size'] - scaler.mean_[4]) / scaler.scale_[4]
    for col in categorical_cols:
        for category in df_client_encoded.columns:
            if category.startswith(col + '_'):
                processed_client[category] = 1 if category.endswith(str(new_client_data[col])) else 0
    return np.array([processed_client[col] for col in client_feature_cols])
def get_top_similar_clients(new_client_vector, top_n=50):
    new_client_embedding = base_network.predict(np.array([new_client_vector]))
    sim_scores = cosine_similarity(new_client_embedding, client_embeddings)[0]
    top_indices = np.argsort(-sim_scores)[:top_n]
    top_similar_clients = [client_ids[i] for i in top_indices]
    return top_similar_clients
new_client_vector = preprocess_new_client(new_client_data)
top_similar_clients = get_top_similar_clients(new_client_vector, top_n=50)
agent_success = (
    df_policies[df_policies['securityno'].isin(top_similar_clients)]
    .groupby('agntnum')
    .size()
    .reset_index(name='match_count')
)
df_agents_subset = df_agents[['agntnum',
                              'pct_SX1_male', 'pct_SX2_female',
                              'pct_AG01_lt20', 'pct_AG02_20to24', 'pct_AG03_25to29',
                              'pct_AG04_30to34', 'pct_AG05_35to39', 'pct_AG06_40to44',
                              'pct_AG07_45to49', 'pct_AG08_50to54', 'pct_AG09_55to59','pct_AG10_60up']]
df_train_agents = df_agents_subset.merge(agent_success, on='agntnum', how='left').copy()
df_train_agents['match_count'] = df_train_agents['match_count'].fillna(0)
if new_client_data.get('client_female', 0) == 1:
    df_train_agents['gender_score'] = df_train_agents['pct_SX2_female']
else:
    df_train_agents['gender_score'] = df_train_agents['pct_SX1_male']
age = new_client_data.get('age')
if age < 20:
    df_train_agents['age_score'] = df_train_agents['pct_AG01_lt20']
elif age < 25:
    df_train_agents['age_score'] = df_train_agents['pct_AG02_20to24']
elif age < 30:
    df_train_agents['age_score'] = df_train_agents['pct_AG03_25to29']
elif age < 35:
    df_train_agents['age_score'] = df_train_agents['pct_AG04_30to34']
elif age < 40:
    df_train_agents['age_score'] = df_train_agents['pct_AG05_35to39']
elif age < 45:
    df_train_agents['age_score'] = df_train_agents['pct_AG06_40to44']
elif age < 50:
    df_train_agents['age_score'] = df_train_agents['pct_AG07_45to49']
elif age < 55:
    df_train_agents['age_score'] = df_train_agents['pct_AG08_50to54']
elif age < 60:
    df_train_agents['age_score'] = df_train_agents['pct_AG09_55to59']
else:
    df_train_agents['age_score'] = df_train_agents['pct_AG10_60up']

df_agents2 = df_agents[['agntnum','agent_tenure',
       'cnt_converted', 'annual_premium_cnvrt', 'pct_lapsed', 'pct_cancel',
       'pct_inforce', 'pct_prod_1_cnvrt', 'pct_prod_2_cnvrt',
       'pct_prod_3_cnvrt', 'pct_prod_4_cnvrt', 'pct_prod_5_cnvrt',
       'pct_prod_6_cnvrt', 'pct_prod_7_cnvrt', 'pct_prod_8_cnvrt',
       'pct_prod_9_cnvrt','agent_expertise_prod_0', 'agent_expertise_prod_6',
       'agent_expertise_prod_4', 'agent_expertise_prod_9',
       'agent_expertise_prod_2', 'agent_expertise_prod_7',
       'agent_expertise_prod_8']].copy()
df_policies2 = df_policies[['chdrnum', 'agntnum', 'securityno', 'product',
       'flg_lapsed', 'flg_expire', 'cust_age_at_purchase_grp',
       'cust_tenure_at_purchase_grp']].copy()
available_products = {0, 2, 4, 6, 7, 8, 9}
if df_policies2['product'].dtype == object:
    df_policies2['product'] = df_policies2['product'].str.replace('prod_', '', regex=False).astype(int)

def get_top_similar_clients_with_scores(new_client_vector, top_n=50):
    new_client_embedding = base_network.predict(np.array([new_client_vector]))
    sim_scores = cosine_similarity(new_client_embedding, client_embeddings)[0]
    top_indices = np.argsort(-sim_scores)[:top_n]
    top_similar_clients = [client_ids[i] for i in top_indices]
    top_similar_scores = [sim_scores[i] for i in top_indices]
    return top_similar_clients, top_similar_scores

def get_top_k_product_suitability(new_client_data, k=7):
    new_client_encoded = preprocess_new_client(new_client_data)
    new_client_vector = new_client_encoded.astype(np.float32)
    print("New client vector shape:", new_client_vector.shape)
    top_clients, top_scores = get_top_similar_clients_with_scores(new_client_vector, top_n=50)
    weight_map = dict(zip(top_clients, top_scores))
    df_sub = df_policies2[df_policies2['securityno'].isin(top_clients)]
    df_sub = df_sub[df_sub['product'].isin(available_products)].copy()
    df_sub['sim_weight'] = df_sub['securityno'].map(weight_map).fillna(0)
    product_agg = df_sub.groupby('product')['sim_weight'].sum().reset_index()
    total_weight = product_agg['sim_weight'].sum()
    if total_weight > 0:
        product_agg['suitability'] = product_agg['sim_weight'] / total_weight
    else:
        product_agg['suitability'] = 0
    product_agg = product_agg.sort_values('suitability', ascending=False)
    top_products = product_agg.head(k)['product'].tolist()
    suitability_dict = product_agg.set_index('product')['suitability'].to_dict()
    print("DEBUG: Top products:", top_products)
    print("DEBUG: Suitability dict:", suitability_dict)
    return top_products, suitability_dict

top_products, suitability_dict = get_top_k_product_suitability(new_client_data, k=3)
print("Final Top Products for New Client:", top_products)
print("Final Suitability Scores:", suitability_dict)

if 'pct_prod_0_cnvrt' not in df_agents2.columns:
    df_agents2 = df_agents2.copy()
    df_agents2['pct_prod_0_cnvrt'] = 1 - df_agents2[['pct_prod_1_cnvrt', 'pct_prod_2_cnvrt',
                                                     'pct_prod_3_cnvrt', 'pct_prod_4_cnvrt',
                                                     'pct_prod_5_cnvrt', 'pct_prod_6_cnvrt',
                                                     'pct_prod_7_cnvrt', 'pct_prod_8_cnvrt',
                                                     'pct_prod_9_cnvrt']].sum(axis=1)
    df_agents2['pct_prod_0_cnvrt'] = df_agents2['pct_prod_0_cnvrt'].clip(lower=0)

conv_map = {
    0: 'pct_prod_0_cnvrt',
    1: 'pct_prod_1_cnvrt',
    2: 'pct_prod_2_cnvrt',
    3: 'pct_prod_3_cnvrt',
    4: 'pct_prod_4_cnvrt',
    5: 'pct_prod_5_cnvrt',
    6: 'pct_prod_6_cnvrt',
    7: 'pct_prod_7_cnvrt',
    8: 'pct_prod_8_cnvrt',
    9: 'pct_prod_9_cnvrt'
}

exp_map = {
    0: 'agent_expertise_prod_0',
    2: 'agent_expertise_prod_2',
    4: 'agent_expertise_prod_4',
    6: 'agent_expertise_prod_6',
    7: 'agent_expertise_prod_7',
    8: 'agent_expertise_prod_8',
    9: 'agent_expertise_prod_9'
}

def compute_agent_product_rating(new_client_data, k=3, w_exp=0.1, w_conv=0.9):
    top_products, suitability_dict = get_top_k_product_suitability(new_client_data, k=k)
    print("Top suitable products:", top_products)
    print("Suitability scores:", suitability_dict)

    def agent_rating(row):
        total = 0.0
        for p in top_products:
            s = suitability_dict.get(p, 0)
            conv_col = conv_map.get(p)
            conv = row[conv_col] if conv_col in row.index else 0
            exp_col = exp_map.get(p)
            exp = row[exp_col] if (exp_col is not None and exp_col in row.index) else 0
            total += s * (w_exp * exp + w_conv * conv)
        return total

    df_agents2_cp = df_agents2.copy()
    df_agents2_cp['agent_product_rating'] = df_agents2_cp.apply(agent_rating, axis=1)

    min_rating = df_agents2_cp['agent_product_rating'].min()
    max_rating = df_agents2_cp['agent_product_rating'].max()
    if max_rating - min_rating > 0:
        df_agents2_cp['agent_product_rating_norm'] = (df_agents2_cp['agent_product_rating'] - min_rating) / (max_rating - min_rating)
    else:
        df_agents2_cp['agent_product_rating_norm'] = 0.0

    df_result = df_agents2_cp[['agntnum', 'agent_product_rating', 'agent_product_rating_norm']].sort_values(by='agent_product_rating_norm', ascending=False)
    return df_result

df_agent_ratings = compute_agent_product_rating(new_client_data, k=10)
print("Top agents based on product experience rating:")
print(df_agent_ratings.head(10))

In [ ]:
############################################
# SECTION 4: AGENT-CLIENT BACKGROUND SIMILARITY
############################################

marital_map = {'Married': 3, 'Single': 2, 'Divorced': 1, 'Unknown': 0}

df_agents_new = df_agents.copy()
df_clients_new = df_clients.copy()

df_agents_new['agent_marital'] = df_agents_new['agent_marital'].map(marital_map)
df_clients_new['client_marital'] = df_clients_new['client_marital'].map(marital_map)

df_agents_new = df_agents_new.dropna(subset=['agntnum', 'agent_age', 'agent_marital', 'agent_female'])
df_clients_new = df_clients_new.dropna(subset=['securityno', 'client_marital', 'age', 'client_female'])

def create_agent_vector(agent_row):
    return np.array([agent_row['agent_age'], agent_row['agent_female'], agent_row['agent_marital']], dtype=float)

def create_client_vector(client_row):
    return np.array([client_row['age'], client_row['client_female'], client_row['client_marital']], dtype=float)

def compute_agent_client_similarity(new_client_data, df_agents):
    new_client_series = pd.Series(new_client_data)
    if isinstance(new_client_series['client_marital'], str):
        new_client_series['client_marital'] = marital_map.get(new_client_series['client_marital'], 0)
    new_client_vector = create_client_vector(new_client_series)
    similarities = []
    for idx, agent_row in df_agents.iterrows():
        agent_vector = create_agent_vector(agent_row)
        norm_client = np.linalg.norm(new_client_vector)
        norm_agent = np.linalg.norm(agent_vector)
        if norm_client == 0 or norm_agent == 0:
            sim = 0
        else:
            sim = np.dot(new_client_vector, agent_vector) / (norm_client * norm_agent)
        similarities.append(sim)
    df_agents_sim = df_agents.copy()
    df_agents_sim['background_similarity'] = similarities
    df_agents_sim = df_agents_sim.sort_values(by='background_similarity', ascending=False)
    return df_agents_sim[['agntnum','background_similarity']]

df_agent_similarity = compute_agent_client_similarity(new_client_data, df_agents_new)
print("Top agents based on background similarity:")
print(df_agent_similarity.head(10))

In [ ]:
if 'pct_prod_0_cnvrt' not in df_agents2.columns:
    df_agents2 = df_agents2.copy()
    df_agents2['pct_prod_0_cnvrt'] = 1 - df_agents2[['pct_prod_1_cnvrt', 'pct_prod_2_cnvrt',
                                                     'pct_prod_3_cnvrt', 'pct_prod_4_cnvrt',
                                                     'pct_prod_5_cnvrt', 'pct_prod_6_cnvrt',
                                                     'pct_prod_7_cnvrt', 'pct_prod_8_cnvrt',
                                                     'pct_prod_9_cnvrt']].sum(axis=1)
    df_agents2['pct_prod_0_cnvrt'] = df_agents2['pct_prod_0_cnvrt'].clip(lower=0)

conv_map = {
    0: 'pct_prod_0_cnvrt',
    1: 'pct_prod_1_cnvrt',
    2: 'pct_prod_2_cnvrt',
    3: 'pct_prod_3_cnvrt',
    4: 'pct_prod_4_cnvrt',
    5: 'pct_prod_5_cnvrt',
    6: 'pct_prod_6_cnvrt',
    7: 'pct_prod_7_cnvrt',
    8: 'pct_prod_8_cnvrt',
    9: 'pct_prod_9_cnvrt'
}

exp_map = {
    0: 'agent_expertise_prod_0',
    2: 'agent_expertise_prod_2',
    4: 'agent_expertise_prod_4',
    6: 'agent_expertise_prod_6',
    7: 'agent_expertise_prod_7',
    8: 'agent_expertise_prod_8',
    9: 'agent_expertise_prod_9'
}

def compute_agent_product_rating(new_client_data, k=3, w_exp=0.1, w_conv=0.9):
    top_products, suitability_dict = get_top_k_product_suitability(new_client_data, k=k)
    print("Top suitable products:", top_products)
    print("Suitability scores:", suitability_dict)

    def agent_rating(row):
        total = 0.0
        for p in top_products:
            s = suitability_dict.get(p, 0)
            conv_col = conv_map.get(p)
            conv = row[conv_col] if conv_col in row.index else 0
            exp_col = exp_map.get(p)
            exp = row[exp_col] if (exp_col is not None and exp_col in row.index) else 0
            total += s * (w_exp * exp + w_conv * conv)
        return total

    df_agents2_cp = df_agents2.copy()
    df_agents2_cp['agent_product_rating'] = df_agents2_cp.apply(agent_rating, axis=1)

    min_rating = df_agents2_cp['agent_product_rating'].min()
    max_rating = df_agents2_cp['agent_product_rating'].max()
    if max_rating - min_rating > 0:
        df_agents2_cp['agent_product_rating_norm'] = (df_agents2_cp['agent_product_rating'] - min_rating) / (max_rating - min_rating)
    else:
        df_agents2_cp['agent_product_rating_norm'] = 0.0

    df_result = df_agents2_cp[['agntnum', 'agent_product_rating', 'agent_product_rating_norm']].sort_values(by='agent_product_rating_norm', ascending=False)
    return df_result

df_agent_ratings = compute_agent_product_rating(new_client_data, k=10)
print("Top agents based on product experience rating:")
print(df_agent_ratings.head(10))

In [ ]:
############################################
# SECTION 5: FINAL COMBINED MODEL & RANKING
############################################

# First Approach: Classification Model
final_df_classification = df_train_agents.merge(
    df_agent_ratings.set_index('agntnum'), on='agntnum', how='outer'
).merge(
    df_agent_similarity.set_index('agntnum'), on='agntnum', how='outer'
)[['agntnum', 'match_count', 'gender_score', 'age_score', 'agent_product_rating', 'background_similarity']]

final_df_classification['target'] = (final_df_classification['match_count'] > 0).astype(int)

features = ['match_count', 'gender_score', 'age_score', 'agent_product_rating', 'background_similarity']
X = final_df_classification[features]
y = final_df_classification['target']

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

final_model_classification = xgb.XGBClassifier(
    objective='binary:logistic',
    eval_metric='auc',
    use_label_encoder=False,
    random_state=42
)
final_model_classification.fit(X_train, y_train, eval_set=[(X_val, y_val)], verbose=False)

final_df_classification['final_score_classification'] = final_model_classification.predict_proba(X)[:, 1]

# Second Approach: Regression Model
final_df_regression = df_train_agents.merge(
    df_agent_ratings.set_index('agntnum'), on='agntnum', how='outer'
).merge(
    df_agent_similarity.set_index('agntnum'), on='agntnum', how='outer'
)[['agntnum', 'match_count', 'gender_score', 'age_score', 'agent_product_rating', 'background_similarity']]

final_df_regression['target'] = final_df_regression['match_count']
features = ['gender_score', 'age_score', 'agent_product_rating', 'background_similarity']
X = final_df_regression[features]
y = final_df_regression['target']

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)
final_model_regression = xgb.XGBRegressor(
    objective='reg:squarederror',
    learning_rate=0.1,
    n_estimators=100,
    max_depth=5,
    random_state=42
)
final_model_regression.fit(X_train, y_train, eval_set=[(X_val, y_val)], verbose=False)

final_df_regression['final_score_regression'] = final_model_regression.predict(X)

# Merge both models' scores using agntnum
final_df_combined = final_df_classification[['agntnum', 'final_score_classification']].merge(
    final_df_regression[['agntnum', 'final_score_regression']], on='agntnum'
)

# Compute the combined score (sum of both models' scores)
final_df_combined['final_score_combined'] = (
    final_df_combined['final_score_classification'] + final_df_combined['final_score_regression']
)

# Sort by the combined final score
final_df_combined_sorted = final_df_combined.sort_values('final_score_combined', ascending=False)

# Print the top 5 agents based on the combined score
print("Top 5 Agents Based on Combined Model:")
final_df_combined_sorted.head(5)
